In [2]:
import pandas as pd
import datetime as datetime
import time


In [3]:
df = pd.read_csv('76fda9d0-69be-4dd5-8108-0de7907fc5a4.csv')

In [4]:
df = df.sort_values(ascending=False, axis=0, by=['REQUEST_ID'])
df = df.head(1000)
df = df.sample(100)
df = df.sort_values(ascending=False, axis=0, by=['REQUEST_ID'])
df.head()

,REQUEST_ID,CREATED_ON,REQUEST_TYPE,REQUEST_ORIGIN,STATUS,DEPARTMENT,NEIGHBORHOOD,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,FIRE_ZONE,X,Y,GEO_ACCURACY
4,298012,2018-10-29T15:09:00,Building Without a Permit,Website,0,"Permits, Licenses and Inspections",Perry North,1.0,26.0,4.200326e+10,1.0,26.0,1.0,15-Jan,-80.024611,40.491444,APPROXIMATE
18,297998,2018-10-29T14:19:00,Collapsed Catch Basin,Call Center,0,Pittsburgh Water and Sewer Authority,Mount Washington,2.0,19.0,4.200319e+10,5.0,19.0,3.0,1-Apr,-80.014537,40.431848,EXACT
37,297979,2018-10-29T13:28:00,Unpermitted HVAC Work,Control Panel,3,"Permits, Licenses and Inspections",Knoxville,3.0,30.0,4.200330e+10,5.0,30.0,3.0,7-Apr,-79.992628,40.410783,APPROXIMATE
51,297965,2018-10-29T12:50:00,Building Maintenance,Call Center,3,"Permits, Licenses and Inspections",Central Lawrenceville,7.0,9.0,4.200309e+10,2.0,9.0,2.0,5-Mar,-79.959013,40.476684,APPROXIMATE
61,297955,2018-10-29T12:37:00,Potholes,Call Center,0,DPW - Street Maintenance,Larimer,9.0,12.0,4.200312e+10,2.0,12.0,5.0,12-Mar,-79.907931,40.461603,EXACT


In [5]:
df.describe()

,REQUEST_ID,STATUS,COUNCIL_DISTRICT,WARD,TRACT,PUBLIC_WORKS_DIVISION,PLI_DIVISION,POLICE_ZONE,X,Y
count,100.000000,100.000000,99.000000,99.000000,9.900000e+01,99.000000,99.000000,99.000000,99.000000,99.000000
mean,297524.380000,1.200000,4.949495,15.414141,4.200320e+10,2.858586,15.414141,3.555556,-79.971121,40.447044
std,280.639809,1.255292,2.789828,7.977021,1.817819e+05,1.456933,7.977021,1.685607,0.045866,0.025565
min,297008.000000,0.000000,1.000000,1.000000,4.200301e+10,1.000000,1.000000,1.000000,-80.058045,40.384826
25%,297302.750000,0.000000,3.000000,10.000000,4.200310e+10,2.000000,10.000000,2.000000,-80.007677,40.429033
50%,297532.500000,1.000000,5.000000,15.000000,4.200315e+10,3.000000,15.000000,4.000000,-79.977450,40.448990
75%,297752.250000,3.000000,7.000000,20.500000,4.200326e+10,3.000000,20.500000,5.000000,-79.930770,40.464996
max,298012.000000,3.000000,9.000000,32.000000,4.200398e+10,6.000000,32.000000,6.000000,-79.873380,40.497041


In [6]:
# df1 = df[int(df['CREATED_ON'][0:4])] > 2017

In [7]:
df['CREATED_ON'][0:10]

4      2018-10-29T15:09:00
18     2018-10-29T14:19:00
37     2018-10-29T13:28:00
51     2018-10-29T12:50:00
61     2018-10-29T12:37:00
72     2018-10-29T11:57:00
75     2018-10-29T11:52:00
85     2018-10-29T11:23:00
97     2018-10-29T10:55:00
123    2018-10-29T10:14:00
Name: CREATED_ON, dtype: object

In [9]:
x = df['CREATED_ON'][4]
print(x)
date = datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S')
date

2018-10-29T15:09:00


datetime.datetime(2018, 10, 29, 15, 9)

In [10]:
# convert to local time, still ignores daylight savings
date = date - datetime.timedelta(seconds=14400)
date

datetime.datetime(2018, 10, 29, 11, 9)

In [11]:
cols = ['REQUEST_ID', 'WEATHER']
dfwx = pd.DataFrame(columns=cols)  #new dataframe to store results

In [12]:
import pycurl, certifi, json
from io import BytesIO

# https://darksky.net/dev/docs#time-machine-request
dsKey = "4702af8fe8916c618732be6bc6d731f6/";
dsUrl = "https://api.darksky.net/forecast/";
exclude = "?exclude=minutely,hourly,flags";
# dsAPI =  dsUrl + dsKey + dsLat + ',' + dsLon + ',' + Time + exclude
i = 0


In [13]:
for index, row in df.iterrows():
    date = datetime.datetime.strptime(row['CREATED_ON'], '%Y-%m-%dT%H:%M:%S') 
    date -= datetime.timedelta(seconds=14400)
    timestamp = int(date.timestamp())
    dsAPI =  dsUrl + dsKey + str(row['Y']) + ',' + str(row['X']) + ',' + str(timestamp) + exclude
    
    buffer = BytesIO()
    c = pycurl.Curl()
    c.setopt(c.URL, dsAPI)
    c.setopt(c.VERBOSE, True)
    c.setopt(c.CAINFO, certifi.where())
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()

    wx = json.loads(buffer.getvalue().decode('iso-8859-1'))
    buffer.close()
    print (dsAPI, wx)
    if i > 5: 
        break
    else: 
        i = i + 1

https://api.darksky.net/forecast/4702af8fe8916c618732be6bc6d731f6/40.49144409,-80.02461106,1540825740?exclude=minutely,hourly,flags {'latitude': 40.49144409, 'longitude': -80.02461106, 'timezone': 'America/New_York', 'currently': {'time': 1540825740, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0, 'precipProbability': 0, 'temperature': 43.66, 'apparentTemperature': 37.85, 'dewPoint': 37.91, 'humidity': 0.8, 'pressure': 1015.61, 'windSpeed': 10.89, 'windGust': 11.01, 'windBearing': 268, 'cloudCover': 1, 'uvIndex': 2, 'visibility': 10, 'ozone': 327.81}, 'daily': {'data': [{'time': 1540785600, 'summary': 'Mostly cloudy throughout the day.', 'icon': 'partly-cloudy-day', 'sunriseTime': 1540813679, 'sunsetTime': 1540851718, 'moonPhase': 0.67, 'precipIntensity': 0.0038, 'precipIntensityMax': 0.0229, 'precipIntensityMaxTime': 1540789200, 'precipProbability': 0.98, 'precipType': 'rain', 'temperatureHigh': 46.22, 'temperatureHighTime': 1540846800, 'temperatureLow': 38.3, 'temperat

https://api.darksky.net/forecast/4702af8fe8916c618732be6bc6d731f6/nan,nan,1540814220?exclude=minutely,hourly,flags {'code': 400, 'error': 'The given location (or time) is invalid.'}
https://api.darksky.net/forecast/4702af8fe8916c618732be6bc6d731f6/40.43961151,-79.99597161,1540813920?exclude=minutely,hourly,flags {'latitude': 40.43961151, 'longitude': -79.99597161, 'timezone': 'America/New_York', 'currently': {'time': 1540813920, 'summary': 'Overcast', 'icon': 'cloudy', 'precipIntensity': 0.0005, 'precipProbability': 0.05, 'precipType': 'rain', 'temperature': 44.46, 'apparentTemperature': 38.59, 'dewPoint': 40.21, 'humidity': 0.85, 'pressure': 1012.94, 'windSpeed': 11.59, 'windGust': 11.7, 'windBearing': 283, 'cloudCover': 1, 'uvIndex': 0, 'visibility': 9.01, 'ozone': 322.5}, 'daily': {'data': [{'time': 1540785600, 'summary': 'Mostly cloudy throughout the day.', 'icon': 'partly-cloudy-day', 'sunriseTime': 1540813667, 'sunsetTime': 1540851716, 'moonPhase': 0.67, 'precipIntensity': 0.0034

In [ ]:
# with open('cleaned.csv', 'w') as fh:
#     df.to_csv(fh)